In [21]:
import os 
import findspark
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType
from pyspark.sql.functions import from_json
from pyspark.sql.types import StructType, StructField, BooleanType, LongType, IntegerType
os.environ['PYSPARK_SUBMIT_ARGS'] = "--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.1 pyspark-shell"

In [22]:
findspark.init()
findspark.find()

'C:\\Users\\Asus\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python38\\site-packages\\pyspark'

In [5]:
spark = SparkSession.builder\
    .master("local")\
    .appName("kafka-example")\
    .getOrCreate()

:: loading settings :: url = jar:file:/usr/local/Cellar/apache-spark/3.3.1/libexec/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/anastasiabogdanova/.ivy2/cache
The jars for the packages stored in: /Users/anastasiabogdanova/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-5dccbef8-fe81-42eb-94a0-abfc536c1f68;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.3.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.3.1 in central
	found org.apache.kafka#kafka-clients;2.8.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.32 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.2 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.2 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2

22/12/15 15:23:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [13]:
df = spark.readStream.format("kafka").option("kafka.bootstrap.servers", "localhost:29092, localhost:39092").option("subscribe", "topic").load()

In [15]:
df1 = df.withColumn("value", df["value"].cast(StringType()))

In [16]:
display(df1)

DataFrame[key: binary, value: string, topic: string, partition: int, offset: bigint, timestamp: timestamp, timestampType: int]

In [18]:
output = df1.writeStream.outputMode("append").format('parquet').option("checkpointLocation", "./kafka_stream.parquet").option("path", './kafka_stream.parquet').start()

22/12/15 15:24:12 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
22/12/15 15:24:12 WARN StreamingQueryManager: Stopping existing streaming query [id=6f20ba3f-2541-4d5b-9e92-ebd48785304a, runId=c4b1e9d5-ca4a-4ddc-9cd0-b1831309b99b], as a new run is being started.


In [ ]:
output.awaitTermination()